1,2.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-44d6dad9-6268-aa00-6a36-0137b213581d)


### skip

In [ ]:
import shutil
shutil.unpack_archive("/content/drive/MyDrive/TSPNet/dataset/train/ts1.zip","/content/train/")

In [ ]:
shutil.unpack_archive("/content/drive/MyDrive/TSPNet/dataset/train/ts2.zip","/content/train/")

In [ ]:
shutil.unpack_archive("/content/drive/MyDrive/TSPNet/dataset/train/ts3.zip","/content/train/")

In [ ]:
shutil.unpack_archive("/content/drive/MyDrive/TSPNet/dataset/train/ts4.zip","/content/train/")

In [ ]:
shutil.unpack_archive("/content/drive/MyDrive/TSPNet/dataset/train/ts5.zip","/content/train/")

In [ ]:
shutil.unpack_archive("/content/drive/MyDrive/TSPNet/dataset/train/ts6.zip","/content/train/")

In [ ]:
shutil.unpack_archive("/content/drive/MyDrive/TSPNet/dataset/train/ts7.zip","/content/train/")

In [ ]:
import os
print(len(os.listdir('/content/train/')))

7096


In [ ]:
%cd /content/train

/content/train


In [ ]:
!zip -r -q train_dataset.zip /content/train

In [ ]:
!cp -R /content/train/train_dataset.zip /content/drive/MyDrive/TSPNet/dataset/train/

### loop

In [ ]:
%cd /content

/content


In [ ]:
shutil.rmtree('/content/valid')

In [ ]:
!mkdir train

In [ ]:
import shutil
shutil.unpack_archive("/content/drive/MyDrive/TSPNet/dataset/train/ts2.zip","/content/train/")

### continue

In [ ]:
shutil.unpack_archive("/content/drive/MyDrive/TSPNet/code.zip","/content/")

In [ ]:
!mkdir output

In [ ]:
%cd /content/code/

/content/code


In [ ]:
#import torch
#torch.cuda.is_available()

True

In [ ]:
#import shutil
shutil.unpack_archive("/content/drive/MyDrive/SLT-2020/CNN_Dataset/I3D/Attempt-2/i3d-features_1.zip","/content")

In [ ]:
import os
len(os.listdir('/content/content/output2/span=8_stride=2'))

1001

In [ ]:
shutil.copy('/content/drive/MyDrive/SLT-2020/CNN_Dataset/I3D/Attempt-2/FINAL_nslt_2000_iters=5104_top1=32.48_top5=57.31_top10=66.31.pt','/content/')

'/content/FINAL_nslt_2000_iters=5104_top1=32.48_top5=57.31_top10=66.31.pt'

In [ ]:
import os

import numpy as np
import torch
import cv2

from models.pytorch_i3d import InceptionI3d

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

def load_rgb_frames(image_dir, vid, start, num, desired_channel_order='rgb'):
    frames = []
    for i in range(start, start + num):
        img = cv2.imread(os.path.join(image_dir, vid, "images" + str(i).zfill(4) + '.png'))

        if desired_channel_order == 'bgr':
            img = img[:, :, [2, 1, 0]]

        img = (img / 255.) * 2 - 1
        frames.append(img)
    return np.asarray(frames, dtype=np.float32)


def load_all_rgb_frames_from_folder(folder, desired_channel_order='rgb'):
    frames = []

    for fn in sorted(os.listdir(folder), key=lambda x: int(x[6:6+4])):
        img = cv2.imread(os.path.join(folder, fn))
        img = cv2.resize(img, dsize=(224, 224))

        if desired_channel_order == 'bgr':
            img = img[:, :, [2, 1, 0]]

        img = (img / 255.) * 2 - 1
        frames.append(img)
    return np.asarray(frames, dtype=np.float32)


def load_rgb_frames_from_video(vid_path, vid, start, num, resize=(256, 256), desired_channel_order='rgb'):
    vidcap = cv2.VideoCapture(video_path)

    frames = []

    total_frames = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)

    vidcap.set(cv2.CAP_PROP_POS_FRAMES, start)
    for offset in range(min(num, int(total_frames - start))):
        success, img = vidcap.read()

        w, h, c = img.shape
        if w < 226 or h < 226:
            d = 226. - min(w, h)
            sc = 1 + d / min(w, h)
            img = cv2.resize(img, dsize=(0, 0), fx=sc, fy=sc)

        if w > 256 or h > 256:
            img = cv2.resize(img, (math.ceil(w * (256 / w)), math.ceil(h * (256 / h))))

        img = (img / 255.) * 2 - 1

        if desired_channel_order == 'bgr':
            img = img[:, :, [2, 1, 0]]

        frames.append(img)

    return np.asarray(frames, dtype=np.float32)


def extract_features_fullvideo(model, inp, framespan, stride):
    rv = []

    indices = list(range(len(inp)))
    groups = []
    for ind in indices:

        if ind % stride == 0:
            groups.append(list(range(ind, ind+framespan)))

    for g in groups:
        # numpy array indexing will deal out-of-index case and return only till last available element
        frames = inp[g[0]: min(g[-1]+1, inp.shape[0])]

        num_pad = 9 - len(frames)
        if num_pad > 0:
            pad = np.tile(np.expand_dims(frames[-1], axis=0), (num_pad, 1, 1, 1))
            frames = np.concatenate([frames, pad], axis=0)

        frames = frames.transpose([3, 0, 1, 2])

        ft = _extract_features(model, frames)

        rv.append(ft)

    return rv


def _extract_features(model, frames):
    inputs = torch.from_numpy(frames)

    inputs = inputs.unsqueeze(0)

    inputs = inputs.cuda()
    with torch.no_grad():
        ft = model.extract_features(inputs)
    ft = ft.squeeze(-1).squeeze(-1)[0].transpose(0, 1)

    ft = ft.cpu()

    return ft


def run(weight, frame_roots, outroot, inp_channels='rgb'):
    frame_dirs = []

    for root in frame_roots:
        paths = sorted(os.listdir(root))

        frame_dirs.extend([os.path.join(root, path) for path in paths])

    # ===== setup models ======
    i3d = InceptionI3d(400, in_channels=3)

    i3d.replace_logits(2000)
    #i3d.replace_logits(1232)

    print('loading weights {}'.format(weight))
    # i3d.load_state_dict(torch.load(weight))
    i3d.load_state_dict(torch.load(weight))

    i3d.cuda()
    # i3d = nn.DataParallel(i3d)

    print('feature extraction starts.')
    i3d.train(False)  # Set model to evaluate mode

    # ===== extract features ======
    # for framespan, stride in [(4, 2), (16, 8), (32, 16)]:
    for framespan, stride in [(8, 2)]:
    # for framespan, stride in [(16, 8), (32, 16), (64, 32)]:

        outdir = os.path.join(outroot, 'span={}_stride={}'.format(framespan, stride))

        if not os.path.exists(outdir):
            os.makedirs(outdir)


        for ind, dir in enumerate(frame_dirs):
            out_path = os.path.join(outdir, os.path.basename(dir)) + '.pt'

            if os.path.exists(out_path):
                print('{} exists, continue'.format(out_path))
                continue

            frames = load_all_rgb_frames_from_folder(dir, inp_channels)
            features = extract_features_fullvideo(i3d, frames, framespan, stride)


            if ind % 1 == 0:
                print(ind, dir, len(features), features[0].shape)
                #print(features.shape)
                #print('\n')

            torch.save(features, os.path.join(outdir, os.path.basename(dir)) + '.pt',_use_new_zipfile_serialization=False)


if __name__ == "__main__":
    weight = '/content/FINAL_nslt_2000_iters=5104_top1=32.48_top5=57.31_top10=66.31.pt'

    # ======= Extract Features for PHEOENIX-2014-T ========
    frame_roots = [
        '/content/train/'
    ]

    # out = '/home/dongxu/Dongxu/workspace/translation/data/PHOENIX-2014-T/features/i3d-features'
    out = '/content/content/output2/'

    run(weight, frame_roots, out, 'rgb')

loading weights /content/FINAL_nslt_2000_iters=5104_top1=32.48_top5=57.31_top10=66.31.pt
feature extraction starts.
0 /content/train/05December_2011_Monday_heute-7425 56 torch.Size([1, 1024])
1 /content/train/05December_2011_Monday_heute-7426 18 torch.Size([1, 1024])
2 /content/train/05December_2011_Monday_heute-7427 113 torch.Size([1, 1024])
3 /content/train/05December_2011_Monday_heute-7428 25 torch.Size([1, 1024])
4 /content/train/05December_2011_Monday_heute-7429 42 torch.Size([1, 1024])
5 /content/train/05December_2011_Monday_heute-7430 67 torch.Size([1, 1024])
6 /content/train/05December_2011_Monday_heute-7431 91 torch.Size([1, 1024])
7 /content/train/05December_2011_Monday_heute-7432 48 torch.Size([1, 1024])
8 /content/train/05December_2011_Monday_heute-7433 62 torch.Size([1, 1024])
9 /content/train/05December_2011_Monday_heute-7434 66 torch.Size([1, 1024])
10 /content/train/05December_2011_Monday_heute-7435 47 torch.Size([1, 1024])
11 /content/train/05December_2011_Monday_heute

In [ ]:
%cd /content

/content


In [ ]:
import os
print(len(os.listdir("/content/dev")))
print(len(os.listdir("/content/test")))

519
642


In [ ]:
import os
#print(len(os.listdir("/content/output/span=12_stride=2")))
#print(len(os.listdir("/content/output/span=16_stride=2")))
print(len(os.listdir("/content/content/output2/span=8_stride=2")))

2002


In [ ]:
%cd /content/content/output2

/content/content/output2


In [ ]:
!zip -r -q i3d-features12.zip span=8_stride=2/*

In [ ]:
!cp -R /content/content/output2/i3d-features12.zip /content/drive/MyDrive/SLT-2020/CNN_Dataset/I3D/Attempt-2/

In [ ]:
temp = torch.load('/content/output/span=8_stride=2/01April_2010_Thursday_heute-6694.pt')
len(temp)

27

In [ ]:
temp[0].shape

torch.Size([1, 1024])

In [ ]:
temp[1].shape

torch.Size([1, 1024])

In [ ]:
len(os.listdir('/content/train/01April_2010_Thursday_heute-6694'))

53

In [ ]:
import shutil
shutil.unpack_archive('/content/drive/MyDrive/SLT-2020/CNN_Dataset/I3D/i3d-features.zip','/content/temp/')

In [ ]:
for file in sorted(os.listdir('/content/temp/content/output/span=8_stride=2/')):
  st = '/content/temp/content/output/span=8_stride=2/' + file
  #print(file)
  #break
  shutil.copy(st,'/content/output/span=8_stride=2/')